<a href="https://colab.research.google.com/github/thunwaaa/sign_language/blob/main/examples/hand_landmarker/python/hand_landmarker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [19]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hand Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect hand landmarks from images.

## Preparation

Let's start with installing MediaPipe.

In [20]:
!pip install protobuf>=5.26.1,<6.0

/bin/bash: line 1: 6.0: No such file or directory


In [21]:
!pip install opencv-python

In [22]:
!pip install mediapipe --upgrade

In [23]:
!pip install numpy==1.26.0

Then download an off-the-shelf model bundle. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker#models) for more information about this model bundle.

In [24]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

## Visualization utilities

In [25]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

import cv2
import numpy as np
import mediapipe as mp
from mediapipe import solutions

# Setup MediaPipe solutions
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Constants
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54)  # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.multi_hand_landmarks
    handedness_list = detection_result.multi_handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks directly using MediaPipe's drawing utilities
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks.landmark]
        y_coordinates = [landmark.y for landmark in hand_landmarks.landmark]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv2.putText(annotated_image, f"{handedness.classification[0].label}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

## Download test image

Let's grab a test image that we'll use later. The image is from [Unsplash](https://unsplash.com/photos/mt2fyrdXxzk).

In [26]:
#

Optionally, you can upload your own image. If you want to do so, uncomment and run the cell below.

In [27]:
import cv2
import mediapipe as mp
import numpy as np
import time
import os

from google.colab import files
uploaded = files.upload()  # อัปโหลดวิดีโอของคุณ

# เก็บชื่อไฟล์ที่อัปโหลด
input_video_path = list(uploaded.keys())[0]
print(f"Uploaded: {input_video_path}")

# กำหนดชื่อไฟล์ CSV output
output_json_path = input_video_path.split('.')[0] + "_landmarks.json"

Saving 1.2.1.mp4 to 1.2.1 (1).mp4
Uploaded: 1.2.1 (1).mp4


## Running inference and visualizing the results

Here are the steps to run hand landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python) to learn more about configuration options that this solution supports.


In [28]:
# @title
# import urllib.request

# # MediaPipe initialization
# BaseOptions = mp.tasks.BaseOptions
# HandLandmarker = mp.tasks.vision.HandLandmarker
# HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
# VisionRunningMode = mp.tasks.vision.RunningMode

# # Create a hand landmarker instance with the video mode
# options = HandLandmarkerOptions(
#     base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
#     running_mode=VisionRunningMode.VIDEO,
#     num_hands=2)

# # Open the video file
# cap = cv2.VideoCapture(input_video_path)
# if not cap.isOpened():
#     print(f"Error: Could not open video file {input_video_path}")
#     exit()

# # Get video properties
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps = cap.get(cv2.CAP_PROP_FPS)
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# # Create video writer for output
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# with HandLandmarker.create_from_options(options) as landmarker:
#     # Initialize timestamp
#     timestamp = 0
#     frame_count = 0

#     while cap.isOpened():
#         success, frame = cap.read()
#         if not success:
#             print("End of video or error reading frame.")
#             break

#         frame_count += 1
#         # Optional: Print progress
#         if frame_count % 10 == 0:
#             print(f"Processing frame {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")

#         # Convert to RGB (MediaPipe requirement)
#         frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

#         # Process the frame
#         results = landmarker.detect_for_video(mp_image, timestamp)
#         timestamp += 1

#         # Create a black canvas instead of using the original frame
#         canvas = np.zeros((height, width, 3), dtype=np.uint8)

#         # Draw hand landmarks
#         if results.hand_landmarks:
#             for idx, hand_landmarks in enumerate(results.hand_landmarks):
#                 # Get hand label (LEFT or RIGHT)
#                 handedness = results.handedness[idx][0].category_name

#                 # Get center of hand for text placement
#                 x_values = [landmark.x for landmark in hand_landmarks]
#                 y_values = [landmark.y for landmark in hand_landmarks]
#                 center_x = int(sum(x_values) / len(x_values) * width)
#                 center_y = int(sum(y_values) / len(y_values) * height)

#                 # Draw connections with white color
#                 for connection in mp.solutions.hands.HAND_CONNECTIONS:
#                     start_idx = connection[0]
#                     end_idx = connection[1]

#                     start_point = (int(hand_landmarks[start_idx].x * width),
#                                   int(hand_landmarks[start_idx].y * height))
#                     end_point = (int(hand_landmarks[end_idx].x * width),
#                                 int(hand_landmarks[end_idx].y * height))

#                     cv2.line(canvas, start_point, end_point, (255, 255, 255), 2)  # White lines

#                 # Draw landmarks with light blue color
#                 for landmark in hand_landmarks:
#                     landmark_point = (int(landmark.x * width),
#                                      int(landmark.y * height))
#                     cv2.circle(canvas, landmark_point, 5, (255, 200, 0), -1)  # Light blue dots

#                 # Display hand label
#                 color = (255, 100, 100) if handedness == "LEFT" else (100, 100, 255)  # Different colors for left/right
#                 text_position = (center_x, center_y - 30)
#                 cv2.putText(canvas, handedness, text_position,
#                            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

#         # Write the canvas to output video
#         out.write(canvas)

#         # Optional: Display the frame (comment out for faster processing)
#         # cv2.imshow('Processing Video', canvas)
#         # if cv2.waitKey(1) & 0xFF == ord('q'):
#         #     break

# # Release resources
# cap.release()
# out.release()
# cv2.destroyAllWindows()

# print(f"Processing complete. Output saved to {output_video_path}")

# files.download(output_video_path)

In [29]:
import os
import cv2
import json
import mediapipe as mp

def process_video_to_json(input_video_path, output_json_path, gesture_label):
    """
    Process a video and extract hand landmarks to a JSON file

    Args:
        input_video_path: Path to the input video file
        output_json_path: Path to save the output JSON file
        gesture_label: Label for the gesture being performed in the video
    """
    # MediaPipe initialization
    BaseOptions = mp.tasks.BaseOptions
    HandLandmarker = mp.tasks.vision.HandLandmarker
    HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
    VisionRunningMode = mp.tasks.vision.RunningMode

    # Create a hand landmarker instance with the video mode
    options = HandLandmarkerOptions(
        base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
        running_mode=VisionRunningMode.VIDEO,
        num_hands=2)

    # Open the video file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {input_video_path}")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Initialize data structure for JSON
    video_landmarks = {
        'video_id': os.path.splitext(os.path.basename(input_video_path))[0],
        'gesture_label': gesture_label,
        'fps': fps,
        'total_frames': total_frames,
        'frames': []
    }

    with HandLandmarker.create_from_options(options) as landmarker:
        # Initialize timestamp
        timestamp = 0
        frame_count = 0

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            frame_count += 1
            # Optional: Print progress
            if frame_count % 10 == 0:
                print(f"Processing frame {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")

            # Convert to RGB (MediaPipe requirement)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

            # Process the frame
            results = landmarker.detect_for_video(mp_image, timestamp)

            # Calculate timestamp in seconds
            timestamp_seconds = frame_count / fps

            # Frame landmark data
            frame_data = {
                'frame_number': frame_count,
                'timestamp': timestamp_seconds,
                'hands': []
            }

            # Write hand landmarks to JSON
            if results.hand_landmarks:
                for idx, hand_landmarks in enumerate(results.hand_landmarks):
                    # Get hand label (LEFT or RIGHT)
                    handedness = results.handedness[idx][0].category_name

                    # Prepare hand landmarks
                    hand_data = {
                        'hand_type': handedness,
                        'landmarks': []
                    }

                    # Process each landmark
                    for landmark_idx, landmark in enumerate(hand_landmarks):
                        hand_data['landmarks'].append({
                            'landmark_id': landmark_idx,
                            'x': landmark.x,
                            'y': landmark.y,
                            'z': landmark.z
                        })

                    frame_data['hands'].append(hand_data)

            # Add frame data if landmarks were detected
            if frame_data['hands']:
                video_landmarks['frames'].append(frame_data)

            timestamp += 1

    # Release resources
    cap.release()

    # Save to JSON
    with open(output_json_path, 'w', encoding='utf-8') as jsonfile:
        json.dump(video_landmarks, jsonfile, ensure_ascii=False, indent=4)

    print(f"Processing complete. Landmarks saved to {output_json_path}")

# รับชื่อท่าทางจากผู้ใช้
gesture_label = input("กรุณาระบุชื่อท่าทาง (เช่น hello, thank_you, เป็นต้น): ")

# ประมวลผลวิดีโอ
process_video_to_json(input_video_path, output_json_path, gesture_label)

# ดาวน์โหลดไฟล์ JSON
files.download(output_json_path)

กรุณาระบุชื่อท่าทาง (เช่น hello, thank_you, เป็นต้น): hi
Processing frame 10/217 (4.6%)
Processing frame 20/217 (9.2%)
Processing frame 30/217 (13.8%)
Processing frame 40/217 (18.4%)
Processing frame 50/217 (23.0%)
Processing frame 60/217 (27.6%)
Processing frame 70/217 (32.3%)
Processing frame 80/217 (36.9%)
Processing frame 90/217 (41.5%)
Processing frame 100/217 (46.1%)
Processing frame 110/217 (50.7%)
Processing frame 120/217 (55.3%)
Processing frame 130/217 (59.9%)
Processing frame 140/217 (64.5%)
Processing frame 150/217 (69.1%)
Processing frame 160/217 (73.7%)
Processing frame 170/217 (78.3%)
Processing frame 180/217 (82.9%)
Processing frame 190/217 (87.6%)
Processing frame 200/217 (92.2%)
Processing frame 210/217 (96.8%)
Processing complete. Landmarks saved to 1_landmarks.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Data Processing

รวบรวมข้อมูลจากทุกไฟล์ JSONในโฟลเดอร์เพื่อเพิ่มความหลากหลายและความครอบคลุมของข้อมูล

In [31]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def preprocess_sign_language_data(data_directory):
    """
    Preprocess sign language data from JSON files

    Args:
        data_directory: Path to directory containing JSON files

    Returns:
        X: Processed feature data
        y: Corresponding labels
    """
    # เก็บข้อมูลทั้งหมด
    all_landmarks = []
    all_labels = []

    # วนลูปผ่านโฟลเดอร์คำศัพท์
    for gesture_folder in os.listdir(data_directory):
        folder_path = os.path.join(data_directory, gesture_folder)

        # ข้ามถ้าไม่ใช่โฟลเดอร์
        if not os.path.isdir(folder_path):
            continue

        # วนลูปผ่านไฟล์ JSON
        for json_file in os.listdir(folder_path):
            if json_file.endswith('.json'):
                file_path = os.path.join(folder_path, json_file)

                # โหลดข้อมูล JSON
                with open(file_path, 'r') as f:
                    video_data = json.load(f)

                # สกัดคุณลักษณะ
                processed_landmarks = process_video_landmarks(video_data)

                # เพิ่มข้อมูล
                all_landmarks.append(processed_landmarks)
                all_labels.append(gesture_folder)

    # แปลง list เป็น numpy array
    X = np.array(all_landmarks)

    # Encode labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(all_labels)

    return X, y, label_encoder

def process_video_landmarks(video_data):
    """
    แปลงข้อมูลแลนด์มาร์คจาก JSON เป็นชุดข้อมูลที่เหมาะสำหรับโมเดล

    Args:
        video_data: ข้อมูล JSON ของวิดีโอ

    Returns:
        processed_landmarks: อาร์เรย์ของแลนด์มาร์ค
    """
    # เลือกเฟรมที่มีการตรวจจับมือ
    hand_frames = [frame for frame in video_data['frames'] if frame['hands']]

    # เลือกเฟรมทั้งหมด (หรือจำกัดจำนวนเฟรม)
    selected_frames = hand_frames[:30]  # จำกัดที่ 30 เฟรม

    # เตรียมอาร์เรย์เก็บแลนด์มาร์ค
    landmarks_sequence = []

    for frame in selected_frames:
        # สำหรับแต่ละมือในเฟรม
        frame_landmarks = []
        for hand in frame['hands']:
            # สกัด x, y, z ของแต่ละจุด
            hand_landmarks = [
                [landmark['x'], landmark['y'], landmark['z']]
                for landmark in hand['landmarks']
            ]
            frame_landmarks.extend(hand_landmarks)

        # padding หากมีจุดไม่ครบ
        while len(frame_landmarks) < 42:  # 21 จุด * 2 มือ
            frame_landmarks.append([0, 0, 0])

        landmarks_sequence.append(frame_landmarks[:42])

    # padding sequence ให้มีความยาวคงที่
    while len(landmarks_sequence) < 30:
        landmarks_sequence.append([[0, 0, 0]] * 42)

    return np.array(landmarks_sequence)

# ตัวอย่างการใช้งาน
data_directory = r"C:\Users\sitta\Documents\project\hi"
print("Checking if path exists:", os.path.exists(data_directory))

X, y, label_encoder = preprocess_sign_language_data(data_directory)

# แบ่งข้อมูล
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Shape of training data:", X_train.shape)
print("Unique labels:", label_encoder.classes_)

Checking if path exists: False


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\sitta\\Documents\\project\\hi'